In [1]:
%pip install scipy
%pip install plotly
%pip install pandas

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd
import plotly.io as pio
import plotly.graph_objects as go
from scipy.cluster.hierarchy import linkage, leaves_list


def create_heatmap_without_sample_clustering(df, color='RdBu_r'):
    """根据基因表达矩阵创建热图

    Args:
        df (pd.DataFrame): 基因表达矩阵
        color (str, optional): 颜色. Defaults to 'RdBu_r'.
    """

    # 第一列改名为gene_id，并设置为索引
    df = df.rename(columns={df.columns[0]: 'gene_id'})
    df = df.set_index('gene_id')

    # 数据Z-score标准化,axis=0表示按列标准化
    df_normalized = df.apply(lambda x: (x - x.mean()) / x.std(), axis=0)
    df_normalized = df_normalized.clip(-3, 3)

    # 对基因（行）进行层次聚类
    gene_linkage = linkage(df_normalized, method='average')
    gene_order = leaves_list(gene_linkage)

    # # 对样本（列）进行层次聚类
    # sample_linkage = linkage(df_normalized.T, method='average')
    # sample_order = leaves_list(sample_linkage)

    # 重排数据矩阵
    df_clustered = df_normalized.iloc[gene_order]
    # df_clustered = df_normalized.iloc[gene_order,sample_order]

    # 创建热图
    fig = go.Figure(data=go.Heatmap(
        z=df_clustered,
        x=df_clustered.columns,
        y=df_clustered.index,
        colorscale=color,
        hovertemplate='Sample: %{x}<br>Gene: %{y}<br>Exp: %{z:.3f}<extra></extra>'
    ))

    # 设置轴标签
    fig.update_layout(
        xaxis_title="Sample",
        yaxis_title="Gene",
        width=1200,
        height=900,
    )

    # 方案1:将fig对象转为json
    # fig_json = pio.to_json(fig)
    # return fig_json

    # 方案2:将fig转为html格式，返回html代码
    # fig_html = pio.to_html(fig, full_html=False, include_plotlyjs=False)        
    # return fig_html

    # 方案3:将fig转为html格式，保存为html文件
    # fig.write_html("./output-file/heatmap.html")
    # return "heatmap.html"

    return fig


# 加载数据
df = pd.read_csv('./Gene_Expression_Matrix.tsv',sep='\t').copy()

# 测试
create_heatmap_without_sample_clustering(df)